In [3]:
import requests
import pandas as pd
import numpy as np
import time 
import urllib.parse
import chardet
import re
from bs4 import BeautifulSoup
#from tqdm import tqdm_notebook as tqdm

In [4]:
pip install chardet

Note: you may need to restart the kernel to use updated packages.


In [5]:
#ディレクトリを指定
import os
os.getcwd()
# os.chdir('/content/drive/MyDrive/M&Aオンライン')

'/Users/maedakesuke/Projects/notebooks'

In [6]:
#お問い合わせフォームを取得する関数を定義
def get_contact_form_url(company_name, representative_name, prefecture):
    print(f"Processing: {company_name}, {representative_name}, {prefecture}")  # ログを追加
    #検索する会社情報
    company = f"[{company_name},{representative_name},{prefecture}]"
    
    # 検索クエリを作成
    query = f"{company} ホームページ"
    
    print("Before Google search") 
    # Google検索結果の取得
    url = f"https://www.google.com/search?q={query}"
    response = requests.get(url)
    response.raise_for_status()

    print("After Google search")  # ログを追加
    # HTMLの解析
    soup = BeautifulSoup(response.text, "html.parser")

    div_tags = soup.find_all('div', class_="egMi0 kCrYT")
    if not div_tags:
        return None

    element = div_tags[0].find_all('a')[0]

    link = element.get('href')

    # リンクをデコード
    # '/url?q=' の後の部分を抽出
    start_index = link.find('/url?q=') + len('/url?q=')
    end_index = link.find('&', start_index)
    decoded_url = urllib.parse.unquote(link[start_index:end_index])
    url = decoded_url
    
    try:
        response = requests.get(url)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        e = ''

    # HTMLの解析
    soup = BeautifulSoup(response.text, "html.parser")
    # hrefに「contact」を含む<a>タグを検索
    contact_links = soup.find_all('a', href=lambda href: href and ('contact' in href or 'inquiry' in href or 'お問合わせ' in href))
   
    if not contact_links:
        contact_links = soup.find_all('a', text='お問い合わせ')

    # お問い合わせフォームのURLを取得
    contact_forms = set()
    for link in contact_links:
        if 'href' in contact_links:
            contact_form = link['href']
        else :
            contact_form = ''
            
        if contact_form.startswith('http'):
            contact_form = contact_form
        elif 'contact' in contact_form or 'inquiry' in contact_form:
            contact_form = decoded_url + contact_form
            
        contact_forms.add(contact_form)
    
    return decoded_url,list(contact_forms)

In [7]:
# ============================================!ファイル名（name = NST紹介先一覧）を更新!==============================================
#紹介先一覧を読み込み
name = 'ECO紹介先一覧 (2)'
df = pd.read_csv(f'{name}.csv',encoding = 'cp932')
df_copy = df
df_copy

FileNotFoundError: [Errno 2] No such file or directory: 'ECO紹介先一覧 (2).csv'

In [11]:
# 新しい列を追加して関数を適用
df_copy['contact_form'] = df_copy.apply(lambda row: get_contact_form_url(row['買手企業名: 会社名'], row['買手企業名: 代表者氏名'],row['買手企業名: 都道府県']), axis=1)
df_copy['会社HP'] = df_copy['contact_form'].str[0]
# df_copy['問い合わせフォーム'] = df_copy['contact_form'].str[1].str[0]

Processing: 株式会社プロスペクト, 　, 東京都
Before Google search
After Google search
Processing: 株式会社Ａ－スタイル, 遠藤　博, 埼玉県
Before Google search
After Google search
Processing: 株式会社ＲＶＨ, 荻野　善之, 東京都
Before Google search
After Google search
Processing: 燦キャピタルマネージメント株式会社, 清末　隆宏, 大阪府
Before Google search
After Google search
Processing: 株式会社アキュラホーム東京中央, 細淵　直樹, 東京都
Before Google search
After Google search
Processing: 株式会社キャッツ, 村上　幸栄, 東京都
Before Google search
After Google search
Processing: フジコーポレーション株式会社, 西村　英二, 東京都
Before Google search
After Google search
Processing: 株式会社リディアルスタッフ, 山口　誠, 東京都
Before Google search
After Google search
Processing: 株式会社Ａｓｔｒｏｎｅｒｇｙ　Ｊａｐａｎ, 崔　宰熏, 東京都
Before Google search
After Google search
Processing: ダイキンエアテクノ株式会社, 桒山　隆, 東京都
Before Google search
After Google search
Processing: いちごオフィスリート投資法人, 福永　隆明, 東京都
Before Google search
After Google search
Processing: 白井石油株式会社, 白井　豊, 茨城県
Before Google search
After Google search
Processing: 株式会社スマートテック, 小寺　雄三, 茨城県
Before Google search
After Googl

In [12]:
# 新しい列を追加して関数を適用
df_copy['contact_form'] = df_copy.apply(lambda row: get_contact_form_url(row['会社名'], row['代表者氏名'],row['都道府県']), axis=1)
df_copy['会社HP'] = df_copy['contact_form'].str[0]
df_copy['問い合わせフォーム'] = df_copy['contact_form'].str[1].str[0]

KeyError: '会社名'

In [13]:
df_copy = df_copy[['買手企業名: 会社名', '買手企業名: 上場/未上場', '買手企業名: 住所','買手企業名: 都道府県','会社HP','買手企業名: 財務情報1_時期(最新)',
       '買手企業名: 財務情報1_売上(千円)（最新）', '買手企業名: 財務情報1_純利益(千円)(最新)', '買手企業名: 事業概要', '買手企業名: TSR備考',
       '買手企業名: 顧客担当者 登録有無', '買手企業名: ニーズレコード件数', '紹介先: 作成日', '買手企業名: 企業情報 ID',
       '紹介先: ID', '買手企業名: 郵便番号', 
       '買手企業名: 電話', '買手企業名: 代表者役職名', '買手企業名: 代表者氏名', '買手企業名: 代表者氏名(姓)',
       '買手企業名: 代表者氏名(名)', '買手企業名: 代表者住所','買手企業名: 最終更新日', '買手企業名: 最終活動日','買手企業名: 最終手紙送付日時（買手）', 
                   '買手企業名: 累積info送付件数（買手）','買手企業名: 最終info送付日時（買手）', '最終架電日時']]
df_copy.to_csv(f'{name}info送付用_test.csv',encoding = 'cp932')
df_copy

,買手企業名: 会社名,買手企業名: 上場/未上場,買手企業名: 住所,買手企業名: 都道府県,会社HP,買手企業名: 財務情報1_時期(最新),買手企業名: 財務情報1_売上(千円)（最新）,買手企業名: 財務情報1_純利益(千円)(最新),買手企業名: 事業概要,買手企業名: TSR備考,...,買手企業名: 代表者氏名,買手企業名: 代表者氏名(姓),買手企業名: 代表者氏名(名),買手企業名: 代表者住所,買手企業名: 最終更新日,買手企業名: 最終活動日,買手企業名: 最終手紙送付日時（買手）,買手企業名: 累積info送付件数（買手）,買手企業名: 最終info送付日時（買手）,最終架電日時
0,株式会社プロスペクト,未上場,東京都渋谷区千駄ヶ谷１－３０－８/ダヴィンチ千駄ヶ谷,東京都,https://prospect.co.jp/,NaN,NaN,NaN,NaN,NaN,...,,NaN,NaN,NaN,2023/11/06,2023/10/16,NaN,NaN,NaN,2023/11/06
1,株式会社Ａ－スタイル,未上場,埼玉県川口市南鳩ヶ谷１-１６-４,埼玉県,https://a-style.ne.jp/company/profile/,2022/07/31,1493672.0,29903.0,外壁工事業，屋根工事業，太陽光発電システム工事業，不動産賃貸業,2022年7月期はコロナ禍や半導体不足の影響により減収となり、建設コスト高や内部経費負担で大...,...,遠藤 博,遠藤,博,〒334-0013埼玉県川口市南鳩ヶ谷4-13-14,2023/10/24,2023/10/24,NaN,NaN,2023/10/19,2023/11/06
2,株式会社ＲＶＨ,上場,東京都港区赤坂８-５-２８,東京都,https://rvh.jp/corporate/data/,2022/03/31,23400.0,-141295.0,純粋持株会社（１００％）,システム開発事業を主力とする持株会社。2022年3月期連結はIT投資需要の回復も、前年3月の...,...,荻野 善之,荻野,善之,〒132-0025東京都江戸川区松江1ー23-14ー401,2023/10/20,2023/10/19,NaN,NaN,2023/10/19,2023/11/06
3,燦キャピタルマネージメント株式会社,上場,大阪府大阪市淀川区西中島７-５-２５新大阪ドイビル５Ｆ,大阪府,https://sun-capitalmanagement.co.jp/wp-content...,2022/03/31,56949.0,-1239337.0,投資事業，アセットマネージメント事業，その他の事業,グループで不動産業等を展開。2023年3月期第3四半期連結業績は、旺盛な不動産投資意欲を背景...,...,清末 隆宏,清末,隆宏,〒102-0082東京都千代田区一番町3-2メイフラット4F,2023/10/25,2023/10/25,NaN,NaN,NaN,2023/11/06
4,株式会社アキュラホーム東京中央,未上場,東京都武蔵野市吉祥寺南町１-３１-２七井ビル１Ｆ,東京都,https://iiyeah-tateru.jp/shops/3314,2022/02/28,5480817.0,125129.0,建物の築造，建築の設計管理及び請負，不動産の売買等,アキュラホームグループの西東京エリア部門として、木造住宅建築工事等を手掛ける。2022年2月...,...,細淵 直樹,細淵,直樹,NaN,2023/11/06,2023/10/26,NaN,NaN,2023/10/04,2023/11/06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1010,大一建設株式会社,未上場,群馬県北群馬郡吉岡町南下２５３,群馬県,https://dai-iti.com/company/theme/,2022/10/31,697484.0,17652.0,不動産業（５０．８％），建築工事業（４９．２％）,渋川・吉岡地区では相応の実績及び知名度を誇る不動産・建築工事業。直近期は不動産事業が伸び悩み...,...,大沢 豪,大沢,豪,〒370-3604群馬県北群馬郡吉岡町南下248-1,2023/10/13,2023/03/17,NaN,NaN,NaN,2023/11/06
1011,西毛建設株式会社,未上場,群馬県甘楽郡下仁田町川井１０９-１,群馬県,https://db.salesnow.jp/companies/6070001012049,2022/06/30,340000.0,NaN,土木工事，とび・土工工事，鋼構造物工事，不動産事業他（１．９％）,下仁田町老舗の建設業者で、土木に関わる公共工事により受注基盤を形成。直近期は前年並の業績確保...,...,橳島 幸正,橳島,幸正,〒370-2606群馬県甘楽郡下仁田町川井109-2,2023/10/13,NaN,NaN,NaN,NaN,2023/11/06
1012,プリンスランド興産株式会社,未上場,群馬県吾妻郡嬬恋村大字大前字細原２２７７,群馬県,https://www.athome.co.jp/ahst/princeland.html,2022/03/31,417281.0,41699.0,別荘管理（６５％），不動産代理・仲介（３０％），貸ビル経営（５％）,不動産管理の秀栄興産出資の関連企業。「軽井沢プリンスランド」の管理・運営を手掛け堅実経営で収...,...,飯塚 淳之,飯塚,淳之,NaN,2023/10/13,NaN,NaN,NaN,NaN,2023/11/06
1013,丸栄建設株式会社,未上場,群馬県吾妻郡嬬恋村芦生田２２３-１,群馬県,https://www.maruei-art.co.jp/profile/,2021/09/30,537137.0,1047.0,建築一式工事（８８．７％），不動産取引，建築設計,嬬恋村内に拠点を置く一級建築士事務所と不動産業を併業する建築工事業者。期間損益維持の経営を続...,...,丸山 博文,丸山,博文,〒377-1523群馬県吾妻郡嬬恋村芦生田495-1,2023/10/13,2023/02/20,NaN,NaN,NaN,2023/11/06
